In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/DataScience/plate_project_group09 

/content/drive/.shortcut-targets-by-id/1Vt0fOz_0sVJoPLSXwaaNYYz64X3oIxL9/DataScience/plate_project_group09


In [ ]:
!cd /content && unzip '/content/drive/MyDrive/DataScience/plate_project_group09/datasets/character.zip'

In [4]:
import os
import random
import numpy as np
from PIL import Image
from tensorflow.keras.utils import Sequence
from data_generator import parse_annotation_xml

class DataGenerator(Sequence):
  def __init__(self, batch_size, data_folder, label_map):
    self.batch_size = batch_size
    self.data_folder = data_folder
    self.list_images, _ = parse_annotation_xml(os.path.join(self.data_folder,'anns'),os.path.join(self.data_folder,'images'))

    self.label_map = label_map
    self.num_samples = len(self.list_images)
    self.indices = np.random.permutation(self.num_samples)

  def on_epoch_end(self):
    self.indices = np.random.permutation(self.num_samples)

  def __len__(self):
    return int(self.num_samples / self.batch_size)

  def __getitem__(self, index):
    batch_indices = self.indices[index*self.batch_size: (index + 1)*self.batch_size]
    list_char_images = []
    list_char_labels = []
    for idx in batch_indices:
      image_info = self.list_images[idx]
      image = Image.open(image_info['filename'])
      for ibox in range(4):
        box = random.choice(image_info['object'])
        width = box['xmax'] - box['xmin']
        height = box['ymax'] - box['ymin']
        xmin = box['xmin'] + np.random.uniform(-0.2, 0.2) * width
        ymin = box['ymin'] + np.random.uniform(-0.2, 0.2) * height
        xmax = box['xmax'] + np.random.uniform(-0.2, 0.2) * width
        ymax = box['ymax'] + np.random.uniform(-0.2, 0.2) * height
        
        char_image = np.array(image.crop([xmin, ymin, xmax, ymax]).resize([48, 48]))
        char_label = self.label_map[box["name"]]

        list_char_images.append(char_image)
        list_char_labels.append(char_label)
    return np.array(list_char_images), np.array(list_char_labels)

In [5]:
from tensorflow.keras.layers import Input, Dense, Conv2D, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

class CharClassifier:
  def __init__(self):
    self.labels = [str(idx) for idx in range(10)] + [chr(idx) for idx in range(65, 65+26)]
    self.label_map = {label: idx for idx, label in enumerate(self.labels)}

  def build_model(self):
    input_layer = Input(shape=[48,48,3])
    conv2d_layer_1 = Conv2D(filters=8, kernel_size=3, strides=1, activation='relu')(input_layer)
    conv2d_layer_2 = Conv2D(filters=8, kernel_size=3, strides=2, activation='relu')(conv2d_layer_1)
    conv2d_layer_3 = Conv2D(filters=16, kernel_size=3, strides=1, activation='relu')(conv2d_layer_2)
    conv2d_layer_4 = Conv2D(filters=32, kernel_size=3, strides=2, activation='relu')(conv2d_layer_3)
    conv2d_layer_5 = Conv2D(filters=32, kernel_size=3, strides=1, activation='relu')(conv2d_layer_4)
    conv2d_layer_6 = Conv2D(filters=64, kernel_size=3, strides=2, activation='relu')(conv2d_layer_5)
    reshape_layer = Reshape([3*3*64])(conv2d_layer_6)
    dense_layer = Dense(len(self.labels), activation='softmax')(reshape_layer)
    self.model = Model(input_layer, dense_layer)
    self.model.summary()

    loss = SparseCategoricalCrossentropy()
    metric = SparseCategoricalAccuracy()
    optimizer = Adam(learning_rate=1e-3)

    self.model.compile(loss=loss, optimizer=optimizer, metrics =[metric])
  
  def train(self):
    train_generator = DataGenerator(32,'/content/train/', self.label_map)
    valid_generator = DataGenerator(32, '/content/valid/', self.label_map)

    save_best = ModelCheckpoint(filepath='models/char_classifier.h5', save_best_only=True, verbose=1)
    self.model.fit_generator(
        train_generator, steps_per_epoch=len(train_generator), epochs=500,
        validation_data=valid_generator, validation_steps=len(valid_generator),
        callbacks =[save_best]
    )

classifier = CharClassifier()
classifier.build_model()
classifier.train()
  

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 46, 46, 8)         224       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 8)         584       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 16)        1168      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 32)          4640      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 32)          9248      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 64)          18496 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/500
31/31 [==============================] - 7s 192ms/step - loss: 3.0469 - sparse_categorical_accuracy: 0.2198 - val_loss: 2.3192 - val_sparse_categorical_accuracy: 0.3564

Epoch 00001: val_loss improved from inf to 2.31918, saving model to models/char_classifier.h5
Epoch 2/500
31/31 [==============================] - 6s 188ms/step - loss: 1.9054 - sparse_categorical_accuracy: 0.4516 - val_loss: 1.7829 - val_sparse_categorical_accuracy: 0.5331

Epoch 00002: val_loss improved from 2.31918 to 1.78292, saving model to models/char_classifier.h5
Epoch 3/500
31/31 [==============================] - 6s 189ms/step - loss: 1.4419 - sparse_categorical_accuracy: 0.5970 - val_loss: 1.3041 - val_sparse_categorical_accuracy: 0.6508

Epoch 00003: val_loss improved from 1.78292 to 1.30407, saving model to models/char_classifier.h5
Epoch 4/500
 7/31 [=====>........................] - ETA: 4s - loss: 1.2216 - sparse_categorical_accuracy: 0.6674

KeyboardInterrupt: ignored